In [1]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

from scipy import stats
from scipy.signal import hilbert,convolve,hann
     
from sklearn.linear_model import LinearRegression

In [2]:
import sys

In [3]:
import tensorflow as tf

In [4]:
import keras
import keras.backend as K
from keras.models import Sequential,Model
from keras.layers import LSTM,Dense

Using TensorFlow backend.


In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
pd.set_option('precision',30)

In [7]:
train_df=pd.read_csv('F:\\Qplus\\Kaggle\\train.csv',chunksize=5000000,iterator=True,
                     dtype={'acoustic_data':np.int16,'time_to_failure':np.float32})

In [8]:
def create_many_features(xc,seg_id=0):
    X=pd.DataFrame(index=[seg_id,])
    zc=np.fft.fft(xc)
    
    X.loc[seg_id,'mean']=xc.mean()
    X.loc[seg_id,'std']=xc.std()
    X.loc[seg_id,'max']=xc.max()
    X.loc[seg_id,'min']=xc.min()
    
    #FFT transform values
    realFFT = np.real(zc)
    imagFFT = np.imag(zc)
    X.loc[seg_id, 'Rmean'] = realFFT.mean()
    X.loc[seg_id, 'Rstd'] = realFFT.std()
    X.loc[seg_id, 'Rmax'] = realFFT.max()
    X.loc[seg_id, 'Rmin'] = realFFT.min()
    X.loc[seg_id, 'Imean'] = imagFFT.mean()
    X.loc[seg_id, 'Istd'] = imagFFT.std()
    X.loc[seg_id, 'Imax'] = imagFFT.max()
    X.loc[seg_id, 'Imin'] = imagFFT.min()
    
    X.loc[seg_id, 'Rmean_last_5000'] = realFFT[-5000:].mean()
    X.loc[seg_id, 'Rstd__last_5000'] = realFFT[-5000:].std()
    X.loc[seg_id, 'Rmax_last_5000'] = realFFT[-5000:].max()
    X.loc[seg_id, 'Rmin_last_5000'] = realFFT[-5000:].min()
    X.loc[seg_id, 'Rmean_last_15000'] = realFFT[-15000:].mean()
    X.loc[seg_id, 'Rstd_last_15000'] = realFFT[-15000:].std()
    X.loc[seg_id, 'Rmax_last_15000'] = realFFT[-15000:].max()
    X.loc[seg_id, 'Rmin_last_15000'] = realFFT[-15000:].min()
    
    X.loc[seg_id, 'std_first_50000'] = xc[:50000].std()
    X.loc[seg_id, 'std_last_50000'] = xc[-50000:].std()
    X.loc[seg_id, 'std_first_10000'] = xc[:10000].std()
    X.loc[seg_id, 'std_last_10000'] = xc[-10000:].std()
    
    X.loc[seg_id, 'avg_first_50000'] = xc[:50000].mean()
    X.loc[seg_id, 'avg_last_50000'] = xc[-50000:].mean()
    X.loc[seg_id, 'avg_first_10000'] = xc[:10000].mean()
    X.loc[seg_id, 'avg_last_10000'] = xc[-10000:].mean()
    
    X.loc[seg_id, 'min_first_50000'] = xc[:50000].min()
    X.loc[seg_id, 'min_last_50000'] = xc[-50000:].min()
    X.loc[seg_id, 'min_first_10000'] = xc[:10000].min()
    X.loc[seg_id, 'min_last_10000'] = xc[-10000:].min()
    
    X.loc[seg_id, 'max_first_50000'] = xc[:50000].max()
    X.loc[seg_id, 'max_last_50000'] = xc[-50000:].max()
    X.loc[seg_id, 'max_first_10000'] = xc[:10000].max()
    X.loc[seg_id, 'max_last_10000'] = xc[-10000:].max()
    
    X.loc[seg_id, 'abs_max'] = np.abs(xc).max()
    X.loc[seg_id, 'abs_min'] = np.abs(xc).min()
    X.loc[seg_id, 'avg_diff'] = np.mean(np.diff(xc))
    #X.loc[seg_id, 'avg_diff_rate'] = np.mean(np.nonzero((np.diff(xc) / xc[:-1]))[0]) #seems do not help
    
    X.loc[seg_id, 'q95'] = np.quantile(xc, 0.95)
    X.loc[seg_id, 'q99'] = np.quantile(xc, 0.99)
    X.loc[seg_id, 'q05'] = np.quantile(xc, 0.05)
    X.loc[seg_id, 'q01'] = np.quantile(xc, 0.01)
    
    X.loc[seg_id, 'abs_q95'] = np.quantile(np.abs(xc), 0.95)
    X.loc[seg_id, 'abs_q99'] = np.quantile(np.abs(xc), 0.99)
    X.loc[seg_id, 'abs_q05'] = np.quantile(np.abs(xc), 0.05)
    X.loc[seg_id, 'abs_q01'] = np.quantile(np.abs(xc), 0.01)
    
    X.loc[seg_id, 'trend'] = add_trend_feature(xc)
    X.loc[seg_id, 'abs_trend'] = add_trend_feature(xc, abs_values=True)
    X.loc[seg_id, 'abs_mean'] = np.abs(xc).mean()
    X.loc[seg_id, 'abs_std'] = np.abs(xc).std()
    
    X.loc[seg_id, 'mad'] = xc.mad()
    X.loc[seg_id, 'kurt'] = xc.kurtosis()
    X.loc[seg_id, 'skew'] = xc.skew()
    X.loc[seg_id, 'med'] = xc.median()
    
    X.loc[seg_id, 'Hilbert_mean'] = np.abs(hilbert(xc)).mean()
    X.loc[seg_id, 'Hann_window_mean'] = (convolve(xc, hann(150), mode='same') / sum(hann(150))).mean()
    X.loc[seg_id, 'classic_sta_lta1_mean'] = classic_sta_lta(xc, 500, 10000).mean()
    X.loc[seg_id, 'classic_sta_lta2_mean'] = classic_sta_lta(xc, 5000, 100000).mean()
    X.loc[seg_id, 'classic_sta_lta3_mean'] = classic_sta_lta(xc, 3333, 6666).mean()
    X.loc[seg_id, 'classic_sta_lta4_mean'] = classic_sta_lta(xc, 10000, 25000).mean()
    
    X.loc[seg_id, 'Moving_average_700_mean'] = xc.rolling(window=700).mean().mean(skipna=True)
    X.loc[seg_id, 'Moving_average_1500_mean'] = xc.rolling(window=1500).mean().mean(skipna=True)
    X.loc[seg_id, 'Moving_average_3000_mean'] = xc.rolling(window=3000).mean().mean(skipna=True)
    X.loc[seg_id, 'Moving_average_6000_mean'] = xc.rolling(window=6000).mean().mean(skipna=True)
    
    ewma = pd.Series.ewm
    X.loc[seg_id, 'exp_Moving_average_300_mean'] = (ewma(xc, span=300).mean()).mean(skipna=True)
    X.loc[seg_id, 'exp_Moving_average_3000_mean'] = ewma(xc, span=3000).mean().mean(skipna=True)
    X.loc[seg_id, 'exp_Moving_average_30000_mean'] = ewma(xc, span=6000).mean().mean(skipna=True)
    
    no_of_std = 2
    X.loc[seg_id, 'MA_700MA_std_mean'] = xc.rolling(window=700).std().mean()
    X.loc[seg_id,'MA_700MA_BB_high_mean'] = (X.loc[seg_id, 'Moving_average_700_mean'] + no_of_std * X.loc[seg_id, 'MA_700MA_std_mean']).mean()
    X.loc[seg_id,'MA_700MA_BB_low_mean'] = (X.loc[seg_id, 'Moving_average_700_mean'] - no_of_std * X.loc[seg_id, 'MA_700MA_std_mean']).mean()
    
    X.loc[seg_id, 'MA_400MA_std_mean'] = xc.rolling(window=400).std().mean()
    X.loc[seg_id,'MA_400MA_BB_high_mean'] = (X.loc[seg_id, 'Moving_average_700_mean'] + no_of_std * X.loc[seg_id, 'MA_400MA_std_mean']).mean()
    X.loc[seg_id,'MA_400MA_BB_low_mean'] = (X.loc[seg_id, 'Moving_average_700_mean'] - no_of_std * X.loc[seg_id, 'MA_400MA_std_mean']).mean()
    X.loc[seg_id, 'MA_1000MA_std_mean'] = xc.rolling(window=1000).std().mean()
    
    X.loc[seg_id, 'iqr'] = np.subtract(*np.percentile(xc, [75, 25]))
    X.loc[seg_id, 'q999'] = np.quantile(xc,0.999)
    X.loc[seg_id, 'q001'] = np.quantile(xc,0.001)
    X.loc[seg_id, 'ave10'] = stats.trim_mean(xc, 0.1)
    
    for windows in [10, 100, 1000]:
        x_roll_std = xc.rolling(windows).std().dropna().values
        x_roll_mean = xc.rolling(windows).mean().dropna().values
        
        X.loc[seg_id, 'ave_roll_std_' + str(windows)] = x_roll_std.mean()
        X.loc[seg_id, 'std_roll_std_' + str(windows)] = x_roll_std.std()
        X.loc[seg_id, 'max_roll_std_' + str(windows)] = x_roll_std.max()
        X.loc[seg_id, 'min_roll_std_' + str(windows)] = x_roll_std.min()
        X.loc[seg_id, 'q01_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.01)
        X.loc[seg_id, 'q05_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.05)
        X.loc[seg_id, 'q95_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.95)
        X.loc[seg_id, 'q99_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.99)
        X.loc[seg_id, 'av_change_abs_roll_std_' + str(windows)] = np.mean(np.diff(x_roll_std))
        X.loc[seg_id, 'av_change_rate_roll_std_' + str(windows)] = np.mean(np.nonzero((np.diff(x_roll_std) / x_roll_std[:-1]))[0])
        X.loc[seg_id, 'abs_max_roll_std_' + str(windows)] = np.abs(x_roll_std).max()
        
        X.loc[seg_id, 'ave_roll_mean_' + str(windows)] = x_roll_mean.mean()
        X.loc[seg_id, 'std_roll_mean_' + str(windows)] = x_roll_mean.std()
        X.loc[seg_id, 'max_roll_mean_' + str(windows)] = x_roll_mean.max()
        X.loc[seg_id, 'min_roll_mean_' + str(windows)] = x_roll_mean.min()
        X.loc[seg_id, 'q01_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.01)
        X.loc[seg_id, 'q05_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.05)
        X.loc[seg_id, 'q95_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.95)
        X.loc[seg_id, 'q99_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.99)
        X.loc[seg_id, 'av_change_abs_roll_mean_' + str(windows)] = np.mean(np.diff(x_roll_mean))
        X.loc[seg_id, 'av_change_rate_roll_mean_' + str(windows)] = np.mean(np.nonzero((np.diff(x_roll_mean) / x_roll_mean[:-1]))[0])
        X.loc[seg_id, 'abs_max_roll_mean_' + str(windows)] = np.abs(x_roll_mean).max()
    return X

In [9]:
#generate features for a list of acoustic data
def gen_features_list(acd_list):
    train_features=pd.DataFrame()
    for acd in acd_list:
        train_features=train_features.append(create_many_features(acd),ignore_index=True)
    #print(train_features.shape)
    return train_features

In [10]:
def split_segments(acd_segments,ttf_segments,split_length):
    length=len(acd_segments)
    #split each complete segements into many small segments
    split_segments_index=np.array_split(np.arange(length),length//split_length)
    print('the length of this segments is %d'%len(split_segments_index))
    acd_split=[]
    ttf_split=pd.Series()
    for indices in split_segments_index:
        acd_split.append(pd.Series(acd_segments[indices]))
        ttf_split=ttf_split.append(pd.Series(ttf_segments[indices][-1]),ignore_index=True)
    #print(len(acd_split))
    return acd_split,ttf_split

In [11]:
def add_trend_feature(arr,abs_values=False):
    index=np.arange(len(arr))
    if abs_values:
        arr=np.abs(arr)
    lr=LinearRegression()
    lr.fit(index.reshape(-1,1),arr)
    return lr.coef_[0]

In [ ]:
def classic_sta_lta(x,length_sta,length_lta):
    sta=np.cumsum(x**2)
    sta=np.require(sta,dtype=np.float)
    
    lta=sta.copy()
    
    sta[length_sta:]=sta[length_sta:]-sta[:-length_sta]
    sta/=length_sta
    
    lta[length_lta:]=lta[length_lta:]-lta[:-length_lta]
    lta/=length_lta
    
    sta[:length_sta-1]=0
    
    dtiny=np.finfo(0.0).tiny
    idx=lta<dtiny
    lta[idx]=dtiny
    return sta/lta
    
    
    

In [ ]:

train_X=pd.DataFrame()
train_y=pd.Series()
acd_to_be_extended=np.array([])
ttf_to_be_extended=np.array([])
last=math.inf
split_length=100000
for chunk in train_df:
    acd=chunk.acoustic_data.values
    ttf=chunk.time_to_failure.values
    #split_index=[]   #record the index for segmentation
    split_index=np.array([])
    if ttf[0]>last:
        #split_index.append[0]
        acd_split,ttf_split=split_segments(acd_to_be_extended,ttf_to_be_extended,split_length)      
        train_X=train_X.append(gen_features_list(acd_split),ignore_index=True)
        train_y=train_y.append(ttf_split,ignore_index=True)
        acd_to_be_extended=np.array([])
        ttf_to_be_extended=np.array([])
        
    find_split=ttf[1:]>ttf[:-1]
    #split_index.append(np.where(find_split))
    split_index=np.append(split_index,np.where(find_split))
    length=len(split_index.tolist())
    if length!=0:     #which means a segment split exists
        print(length)
        print(split_index)
        acd_to_be_extended=np.append(acd_to_be_extended,
                                     acd[:int(split_index[0]+1)])
        ttf_to_be_extended=np.append(ttf_to_be_extended,ttf[:int(split_index[0]+1)])
        acd_split,ttf_split=split_segments(acd_to_be_extended,ttf_to_be_extended,split_length)
        train_X=train_X.append(gen_features_list(acd_split),ignore_index=True)
        train_y=train_y.append(ttf_split,ignore_index=True)
        #print(train_X.describe())
        #print(train_y.describe())
        acd_to_be_extended=np.array([])
        ttf_to_be_extended=np.array([])
        
        for i in range(length-1):
            acd_to_be_extended=acd[int(split_index[i])+1:int(split_index[i+1])+1]
            ttf_to_be_extended=ttf[int(split_index[i])+1:int(split_index[i+1])+1]
            acd_split,ttf_split=split_segments(acd_to_be_extended,ttf_to_be_extended,split_length)
            train_X=train_X.append(gen_features_list(acd_split),ignore_index=True)
            train_y=train_y.append(ttf_split,ignore_index=True)
            acd_to_be_extended=np.array([])
            ttf_to_be_extended=np.array([])
        acd_to_be_extended=acd[int(split_index[-1]):]
        ttf_to_be_extended=ttf[int(split_index[-1]):]
        
    else:
        acd_to_be_extended=np.append(acd_to_be_extended,acd)
        ttf_to_be_extended=np.append(ttf_to_be_extended,ttf)
    last=ttf[-1]

1
[656573.]
the length of this segments is 56
1
[85877.]
the length of this segments is 444


In [ ]:
train_X.shape

In [ ]:
train_X.describe()

In [ ]:
test_X_seg_ids=pd.read_csv('F:\\Qplus\\Kaggle\\sample_submission.csv',dtype={'time_to_failure':np.float32},index_col='seg_id')

In [ ]:
test_X_seg_ids.head()

In [ ]:
test_X=pd.DataFrame()
for seg_id in test_X_seg_ids.index.values:
    #print(type(seg_id))
    seg_data=pd.read_csv('F:\\Qplus\\Kaggle\\test\\'+seg_id+'.csv',dtype={'acoustic_data':np.float32})
    test_X=test_X.append(create_many_features(seg_data.acoustic_data,seg_id=seg_id),ignore_index=True)

In [ ]:
test_X.shape

In [ ]:
Merged_X=pd.concat((train_X,test_X),ignore_index=True)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler.fit(Merged_X)
train_X_scaled=scaler.transform(train_X)

In [ ]:
test_X_scaled=scaler.transform(test_X)

In [ ]:
train_X_transformed=train_X_scaled.reshape(train_X_scaled.shape[0],train_X_scaled.shape[1],1)

In [ ]:
train_X_transformed.shape

In [ ]:
LSTM_model=Sequential()
LSTM_model.add(LSTM(100,input_shape=(train_X_transformed.shape[1],train_X_transformed.shape[2])))
LSTM_model.add(Dense(1))

In [ ]:
LSTM_model.compile(optimizer='adam',loss='mae')
history=LSTM_model.fit(train_X_transformed,train_y,epochs=200,batch_size=64,verbose=True)

In [ ]:
test_X_scaled_transformed = test_X_scaled.reshape(test_X_scaled.shape[0],
                                                  test_X_scaled.shape[1], 1)
y_predict = LSTM_model.predict(test_X_scaled_transformed)

In [ ]:
submission = pd.read_csv('F:\\Qplus\\Kaggle\\sample_submission.csv', index_col='seg_id')

In [ ]:
submission['time_to_failure'] = y_predict

In [ ]:
submission.to_csv('F:\\Qplus\\Kaggle\\submission.csv',index=True)

In [ ]:
#1 we will use LSTM model first to predict time_to_failure
# monitored criterion should in the metrics list for model's compile function
callbacks_earlystopping=[keras.callbacks.EarlyStopping(patience=20,monitor='val_loss')]

In [ ]:
LSTM_model=Sequential()
LSTM_model.add(LSTM(100,input_shape=(train_X_transformed.shape[1],train_X_transformed.shape[2])))
LSTM_model.add(Dense(1))

In [ ]:
LSTM_model.compile(optimizer='adam',loss='mean_squared_error',metrics=['mae'])
history=LSTM_model.fit(train_X_transformed,train_y,epochs=500,batch_size=64,
                       validation_split=0.25,callbacks=callbacks_earlystopping,verbose=True)

In [ ]:
history.history.keys()

In [ ]:
plt.plot(history.history['val_loss'])
plt.plot(history.history['loss'])
plt.legend(['val_loss','loss'])
plt.xlabel('epoch')
plt.ylabel('loss')

In [ ]:
plt.show()

In [ ]:
plt.plot(history.history['val_mean_absolute_error'])
plt.plot(history.history['mean_absolute_error'])
plt.legend(['val_mae','mae'])
plt.xlabel('epoch')
plt.ylabel('mae')

In [ ]:
plt.show()

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca=PCA()
pca.fit(train_X)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
#obviously only the 

In [ ]:
pca=PCA()
pca.fit(train_X_scaled)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
#from the pca analysis, we will only use the first 32 components
pca=PCA(n_components=32)
X_dim_reduced=pca.fit_transform(train_X_scaled)

In [ ]:
X_dim_reduced_transformed=X_dim_reduced.reshape(X_dim_reduced.shape[0],X_dim_reduced.shape[1],1)

In [ ]:
#2 we will use dimensions reduced by PCA, then apply LSTM model to predict time_to_failure
# monitored criterion should in the metrics list for model's compile function
callbacks_earlystopping=[keras.callbacks.EarlyStopping(patience=20,monitor='val_loss')]
LSTM_model=Sequential()
LSTM_model.add(LSTM(100,input_shape=(X_dim_reduced_transformed.shape[1],
                                     X_dim_reduced_transformed.shape[2])))
LSTM_model.add(Dense(1))
LSTM_model.compile(optimizer='adam',loss='mean_squared_error',metrics=['mae'])
history=LSTM_model.fit(X_dim_reduced_transformed,train_y,epochs=500,batch_size=64,
                       validation_split=0.25,callbacks=callbacks_earlystopping,verbose=True)

In [ ]:
plt.plot(history.history['val_loss'])
plt.plot(history.history['loss'])
plt.legend(['val_loss','loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

In [ ]:
test_X_seg_ids=pd.read_csv('F:\\Qplus\\Kaggle\\sample_submission.csv',
                           dtype={'time_to_failure':np.float32},index_col='seg_id')
test_X=pd.DataFrame()
for seg_id in test_X_seg_ids.index.values:
    #print(type(seg_id))
    seg_data=pd.read_csv('F:\\Qplus\\Kaggle\\test\\'+seg_id+'.csv',
                         dtype={'acoustic_data':np.float32})
    test_X=test_X.append(create_many_features(seg_data.acoustic_data,seg_id=seg_id),
                         ignore_index=True)

In [ ]:
scaler=StandardScaler()
scaler.fit(Merged_X)
train_X_scaled=scaler.transform(train_X)
test_X_scaled=scaler.transform(test_X)
test_X_scaled_transformed = test_X_scaled.reshape(test_X_scaled.shape[0],
                                                  test_X_scaled.shape[1], 1)

In [ ]:
# test data should go through the same PCA reduction process
pca=PCA(n_components=32)
test_X_dim_reduced=pca.fit_transform(test_X_scaled)

In [ ]:
test_X_dim_reduced_transformed= test_X_dim_reduced.reshape(test_X_dim_reduced.shape[0],
                                                  test_X_dim_reduced.shape[1], 1)
predict_y=LSTM_model.predict(test_X_dim_reduced_transformed)

In [ ]:
predict_y[:10]

In [ ]:
submission['time_to_failure'] = predict_y
submission.to_csv('F:\\Qplus\\Kaggle\\submission_1.csv',index=True)

In [ ]:
#Use SVR for prediction
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

In [ ]:
svr=SVR(kernel='rbf')
svr.fit(train_X_scaled,train_y)

In [ ]:
svr.predict(test_X)

In [ ]:
svr_parameters={'kernel':['linear','rbf'],
                'gamma':[1e-4, 0.001, 0.01, 'auto'],
                'C':[0.1, 0.2,0.5,1,3,5],
               'epsilon':[0.5,1,2]}

In [ ]:
svr=SVR()
gcv=GridSearchCV(svr,param_grid=svr_parameters,scoring='neg_mean_absolute_error')
gcv.fit(train_X_scaled,train_y)

In [ ]:
gcv.best_score_

In [ ]:
gcv.best_params_

In [ ]:
gcv.best_params_

In [ ]:
svr=SVR()
gcv=GridSearchCV(svr,param_grid=svr_parameters)
gcv.fit(train_X_scaled,train_y)

In [ ]:
gcv.best_params_

In [ ]:
gcv.best_score_

In [ ]:
gcv.best_params_

Obviously the best parameter for gamma is 'auto' (this default parameter means that gamma=1/n_features), so the gridsearch should include the search of 'auto'. 

In [ ]:
gcv.best_score_

In [ ]:
gcv.best_estimator_

In [ ]:
gcv.predict(test_X_scaled)

In [ ]:
submission['time_to_failure'] = gcv.predict(test_X_scaled)
submission.to_csv('F:\\Qplus\\Kaggle\\submission_2.csv',index=True)